In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import os
import sys
# Add project root to PYTHONPATH
project_root = os.getcwd()
if str(project_root) not in sys.path and project_root.split('/')[-1] == 'chest_ct_retrieval':
    sys.path.append(str(project_root))
elif str(project_root) not in sys.path and project_root.split('/')[-1] == 'notebooks':
    sys.path.append(str(os.path.normpath('/'.join(project_root.split('/')[:-1]))))

%pwd
%cd "/dataChivo/jpmunoz/chest_ct_retrieval"


/net/backups/exports/dataChivo/jpmunoz/chest_ct_retrieval


In [2]:
import torch
from config.config import load_config
from utils.seed import set_seed
from training.setup import setup_training_run
from training.environment import configure_environment
from eval.metric_loader import load_metrics
from training.data_setup import load_dataset, create_loaders
from training.model_setup import initialize_model
from training.trainer import Trainer


cuda_available = torch.cuda.is_available()

cfg = load_config("config/base.yaml")

set_seed(cfg["training"]["seed"])

run_dirs = setup_training_run(cfg["paths"]["dr2156"]["triplet_runs"])

checkpoints_dir = run_dirs["checkpoints"]
tensorboard_dir = run_dirs["logs"]

configure_environment(cfg)

train_set, test_set, neg_compatibles = load_dataset(
    cfg["paths"]["dr2156"]["preprocessed_300_int8"], 
    cfg["training"]["seed"], 
    float(cfg["dataset"]["train_frac"]),
    augmentations_arg=cfg["training"]["augmentations"]
)

p_model, p_loss_fn, p_optimizer, p_scheduler = initialize_model(
    embedding_size=int(cfg["model"]["embedding_size"]),
    margin=float(cfg["loss"]["margin"]),
    lr=float(cfg["training"]["optimizer"]["lr"]),
    weight_decay=float(cfg["training"]["optimizer"]["weight_decay"]),
    negative_compatibles_dict=neg_compatibles,
    print_interval=int(cfg["logging"]["log_interval"]),
    cuda=cuda_available
)

loaders = create_loaders(
    train_set,
    test_set,
    cfg["training"]["batch"]["n_classes"],
    cfg["training"]["batch"]["n_samples"],
    cuda_available
)

p_metrics = load_metrics(cfg)

In [3]:
test_set.test_triplets

[[0, np.int64(376), np.int64(253)],
 [1, np.int64(214), np.int64(124)],
 [2, np.int64(120), np.int64(21)],
 [3, np.int64(38), np.int64(9)],
 [4, np.int64(54), np.int64(352)],
 [5, np.int64(286), np.int64(388)],
 [6, np.int64(233), np.int64(242)],
 [7, np.int64(417), np.int64(312)],
 [8, np.int64(146), np.int64(212)],
 [9, np.int64(164), np.int64(400)],
 [10, np.int64(89), np.int64(353)],
 [11, np.int64(294), np.int64(1)],
 [12, np.int64(168), np.int64(372)],
 [13, np.int64(270), np.int64(12)],
 [14, np.int64(339), np.int64(312)],
 [15, np.int64(357), np.int64(85)],
 [16, np.int64(46), np.int64(371)],
 [17, np.int64(4), np.int64(236)],
 [18, np.int64(23), np.int64(135)],
 [19, np.int64(150), np.int64(189)],
 [20, np.int64(136), np.int64(8)],
 [21, np.int64(0), np.int64(46)],
 [22, np.int64(1), np.int64(291)],
 [23, np.int64(218), np.int64(282)],
 [24, np.int64(128), np.int64(244)],
 [25, np.int64(383), np.int64(158)],
 [26, np.int64(236), np.int64(426)],
 [27, np.int64(359), np.int64(40

In [2]:
import torch
from config.config import load_config
from utils.seed import set_seed
from training.setup import setup_training_run
from training.environment import configure_environment
from eval.metric_loader import load_metrics
from training.data_setup import load_dataset, create_loaders
from training.model_setup import initialize_model
from training.trainer import Trainer


cuda_available = torch.cuda.is_available()

cfg = load_config("config/base.yaml")

set_seed(cfg["training"]["seed"])

run_dirs = setup_training_run(cfg["paths"]["dr2156"]["triplet_runs"])

checkpoints_dir = run_dirs["checkpoints"]
tensorboard_dir = run_dirs["logs"]

configure_environment(cfg)

train_set, test_set, neg_compatibles = load_dataset(
    cfg["paths"]["dr2156"]["preprocessed_300_int8"], 
    cfg["training"]["seed"], 
    float(cfg["dataset"]["train_frac"]),
    augmentations_arg=cfg["training"]["augmentations"]
)

p_model, p_loss_fn, p_optimizer, p_scheduler = initialize_model(
    embedding_size=int(cfg["model"]["embedding_size"]),
    margin=float(cfg["loss"]["margin"]),
    lr=float(cfg["training"]["optimizer"]["lr"]),
    weight_decay=float(cfg["training"]["optimizer"]["weight_decay"]),
    negative_compatibles_dict=neg_compatibles,
    print_interval=int(cfg["logging"]["log_interval"]),
    cuda=cuda_available
)

loaders = create_loaders(
    train_set,
    test_set,
    cfg["training"]["batch"]["n_classes"],
    cfg["training"]["batch"]["n_samples"],
    cuda_available
)

p_metrics = load_metrics(cfg)

trainer = Trainer(
    train_loader=loaders["train_triplet"],
    val_loader=loaders["test_triplet"],
    train_eval_loader=loaders["train_eval"],
    val_eval_loader=loaders["test_eval"],
    train_full_loader=loaders["all_triplet_train"],
    val_full_loader=loaders["all_triplet_test"],
    model=p_model,
    loss_fn=p_loss_fn,
    optimizer=p_optimizer,
    scheduler=p_scheduler,
    n_epochs=cfg["training"]["n_epochs"],
    cuda=cuda_available,
    log_interval=cfg["logging"]["log_interval"],
    checkpoint_dir=checkpoints_dir,
    tensorboard_logs_dir=tensorboard_dir,
    train_full_loader_switch=cfg["training"]["train_full_loader_switch"],
    metrics=p_metrics,
    start_epoch=0,
    accumulation_steps=3
)
trainer.fit()


  0%|          | 0/50 [00:00<?, ?it/s]


### EPOCH 1 START ###


  0%|          | 0/138 [00:00<?, ?it/s]

/net/backups/exports/dataChivo/jpmunoz/venvs/proximity/lib/python3.12/site-packages/torchio/data/image.py:248: UserWarning: Using TorchIO images without a torchio.SubjectsLoader in PyTorch >= 2.3 might have unexpected consequences, e.g., the collated batches will be instances of torchio.Subject with 5D images. Replace your PyTorch DataLoader with a torchio.SubjectsLoader so that the collated batch becomes a dictionary, as expected. See https://github.com/TorchIO-project/torchio/issues/1179 for more context about this issue.
  warnings.warn(message, stacklevel=1)



### BATCH 1 OF 138 TRAINING LOSS ###
Batch labels: tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       device='cuda:0')
loss: tensor(0.0477, device='cuda:0', grad_fn=<DivBackward0>)
n_triplets: 180

### BATCH 2 OF 138 TRAINING LOSS ###
Batch labels: tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       device='cuda:0')
loss: tensor(0.0526, device='cuda:0', grad_fn=<DivBackward0>)
n_triplets: 180

### BATCH 3 OF 138 TRAINING LOSS ###
Batch labels: tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       device='cuda:0')
loss: tensor(0.0404, device='cuda:0', grad_fn=<DivBackward0>)
n_triplets: 178

### BATCH 4 OF 138 TRAINING LOSS ###
Batch labels: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       device='cuda:0')
loss: tensor(0.0479, device='cuda:0', grad_fn=<DivBackward0>)
n_triplets: 176

### BATCH 5 OF 138 TRAINING LOSS ###
Batch labels: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 

  0%|          | 0/138 [00:00<?, ?it/s]


### BATCH 1 OF 138 TRAINING LOSS ###
Batch labels: tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       device='cuda:0')
loss: tensor(0.0481, device='cuda:0', grad_fn=<DivBackward0>)
n_triplets: 180

### BATCH 2 OF 138 TRAINING LOSS ###
Batch labels: tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       device='cuda:0')
loss: tensor(0.0490, device='cuda:0', grad_fn=<DivBackward0>)
n_triplets: 178

### BATCH 3 OF 138 TRAINING LOSS ###
Batch labels: tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       device='cuda:0')
loss: tensor(0.0562, device='cuda:0', grad_fn=<DivBackward0>)
n_triplets: 179

### BATCH 4 OF 138 TRAINING LOSS ###
Batch labels: tensor([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       device='cuda:0')
loss: tensor(0.0529, device='cuda:0', grad_fn=<DivBackward0>)
n_triplets: 180

### BATCH 5 OF 138 TRAINING LOSS ###
Batch labels: tensor([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1, 

: 

: 

: 

In [ ]:
del tensor, model, subject  # or any other large variables
import gc
gc.collect()
import torch
torch.cuda.empty_cache()